In [3]:
import pandas as pd
import transformers
import torch
import langdetect

def is_english(text):
    try:
        return langdetect.detect(text) == 'en'
    except:
        return False

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

detector = transformers.AutoModelForSequenceClassification.from_pretrained("TrustSafeAI/RADAR-Vicuna-7B")
tokenizer = transformers.AutoTokenizer.from_pretrained("TrustSafeAI/RADAR-Vicuna-7B")

/home/mike/.pyenv/versions/3.10.0/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mike/.pyenv/versions/3.10.0/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
df = pd.read_csv('2017.csv')

print(df.head())

   Unnamed: 0  rating                                         title  \
0          18       5                                    Five Stars   
1          19       4                                    Four Stars   
2          25       5                       Nice lightweight sleeve   
3          27       4  Did anyone use this for a recessed TV mount?   
4         145       5                                     Wonderful   

                                                text images        asin  \
0       great product. high quality. fits perfectly.     []  B01LQF8B7G   
1           Pretty cool!!!<br /><br />Thanks Amazon.     []  B00ZV9RDKK   
2  I am not sure how much protection this provide...     []  B01976K5B6   
3  I believe the electrical boxes are made especi...     []  B001PL3XJS   
4  Takes beautiful pictures and easy to use. Came...     []  B01D0PVYH4   

  parent_asin                       user_id                timestamp  \
0  B01LQF8B7G  AGKHLEW2SOWHNMFQIJGBECAF7INQ  2017-

In [6]:
# get random sample of 1000 rows

max_length = 512

# filter out rows where 'text' column is empty or not a string

df = df[df['text'].apply(lambda x: isinstance(x, str) and len(x) > 0)]

In [7]:
df['token_count'] = df['text'].apply(lambda x: len(tokenizer(x, truncation=False)['input_ids']))

In [8]:
# filter out rows where 'text' column is longer than max_length

df = df[df['token_count'] <= max_length]

# filter out rows where 'text' column is not in English
print(len(df))

3022630


In [9]:
# sort dataframe descending by 'token_count' column
df = df.sort_values(by='token_count', ascending=False)

In [10]:
df = df.head(20_000)
print(len(df))

20000


In [11]:
df_is_english = df[df['text'].apply(lambda x: is_english(x))]
print(len(df_is_english))

19998


In [12]:
df = df.head(1_000)
print(df.head())

         Unnamed: 0  rating  \
602598       396515       5   
1931583       81143       5   
1198976      931887       3   
576155       102316       5   
2424542      778733       5   

                                                     title  \
602598                                         Great value   
1931583  Even on a older (2009) "big box mart" computer...   
1198976  Nice universal mounting tech, decent keyboard,...   
576155   Amazing value, beast of a gaming machine, grea...   
2424542  Great remote, great iOS app - with note re App...   

                                                      text  \
602598   I've used several zip tie guns at work. I trie...   
1931583  First off lets look what I'm using this in:  A...   
1198976  So the good news is that, unlike several other...   
576155   I play too much Rocket League. I started on PS...   
2424542  Setup:  Samsung 55&quot; Smart TV (I don't usi...   

                                                    images        a

In [13]:
df.to_csv('2017-sample-long.csv', index=False)